# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
database_name = 'uk_food'
if database_name in mongo.list_database_names():
    print(f"The '{database_name}' database exists.")
else:
    print(f"The '{database_name}' database does not exist.")


The 'uk_food' database exists.


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
collection_name = 'establishments'
collection = db[collection_name]
document = collection.find_one()
pprint(document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64a73a98f74728140faf1624'),
 'geocode': {'latitude': Decimal128('51.0838120000000'),
             'longitude': Decimal128('1.19562500000000')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
   

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
penang_flavours = []

In [9]:
# Insert the new restaurant into the collection
new_document = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

insert_result = establishments.insert_one(new_document)

In [10]:
# Check that the new restaurant was inserted
query = {"BusinessName": "Penang Flavours"}
document = establishments.find_one(query)

if document:
    print("Document exists in establishments collection.")
else:
    print("Document does not exist in establishments collection.")

Document exists in establishments collection.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Find the matching documents and retrieve only the specified fields
results = establishments.find(query, {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}).limit(10)

# Print the results
for result in results:
    pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessName": "Penang Flavours"}
update = {"$set": {"BusinessTypeID": 1}}

# Perform the update operation
result = establishments.update_one(query, update)

In [13]:
# Confirm that the new restaurant was updated
# Find the document with BusinessName = "Penang Flavours"
query = {"BusinessName": "Penang Flavours"}
result = establishments.find(query, {"_id": 0, "BusinessTypeID": 1})

# Print the document
document = result[0]
business_type_id = document.get("BusinessTypeID")
print("Penang Flavours")
print("BusinessTypeID:", business_type_id)

Penang Flavours
BusinessTypeID: 1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the number of documents that match the query
count = establishments.count_documents(query)

# Print the count
print("Number of documents with LocalAuthorityName as 'Dover':", count)

Number of documents with LocalAuthorityName as 'Dover': 0


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}

# Delete the documents that match the query
result = establishments.delete_many(query)

# Print the number of documents deleted
print("Number of documents deleted:", result.deleted_count)

Number of documents deleted: 0


In [16]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}

# Count the number of documents that match the query
count = establishments.count_documents(query)

# Print the count
print("Number of documents with LocalAuthorityName as 'Dover':", count)

Number of documents with LocalAuthorityName as 'Dover': 0


In [17]:
# Check that other documents remain with 'find_one'
# Define a query to find a document
query = {"LocalAuthorityName": {"$ne": "Dover"}}

# Find a single document that matches the query
remaining_document = establishments.find_one(query)

# Print the remaining document
print("Remaining document:")
pprint(remaining_document)

Remaining document:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64a73a98f74728140faf1624'),
 'geocode': {'latitude': Decimal128('51.0838120000000'),
             'longitude': Decimal128('1.19562500000000')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establis

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set': {"geocode.longitude": {'$toDecimal': "$geocode.longitude"}}}, { '$set':{"geocode.latitude": {'$toDecimal': "$geocode.latitude"}}}])

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [20]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{'$set': {"RatingValue": {'$toInt': "$RatingValue"}}}])

# Print the number of documents updated
print("Number of documents updated:", establishments.count_documents({}))

Number of documents updated: 38789


In [22]:
# Check that the coordinates and rating value are now numbers
# Find a single document to check the updated fields
document = establishments.find_one({})

# Print the updated fields
print("Updated Coordinates:")
print("Longitude:", document['geocode']['longitude'])
print("Latitude:", document['geocode']['latitude'])
print("Updated RatingValue:", document['RatingValue'])

Updated Coordinates:
Longitude: 1.19562500000000
Latitude: 51.0838120000000
Updated RatingValue: 5
